In [1]:
import os,sys, json
import magic, hashlib, os, traceback
import ntpath
from capstone import *
from capstone.x86 import *

import collections
import traceback
import lief

In [2]:
bin_path = '/home/raisul/DATA/x86_pe_msvc_sample' #'/home/raisul/DATA/x86_O2_d4_mingw32_PE' 

bin_files = [os.path.join(bin_path, f) for f in os.listdir(bin_path) ][0:2]

ground_truth_path ='/home/raisul/ANALYSED_DATA/ghidra_x86_pe_msvc/' # '/home/raisul/ANALYSED_DATA/ghidra_x86_O2_d4_mingw32_PE'  

In [3]:
len(bin_files)

2

In [4]:

def get_ground_truth_ghidra(exe_path, text_section_offset , text_section_len):

    text_sextion_end = text_section_offset + text_section_len
    
    elf_file_name = os.path.basename(exe_path)
    ghidra_file_path = os.path.join(ground_truth_path, elf_file_name.split('.')[0]) + '.json'
    
    with open(ghidra_file_path, "r") as file:
        ghidra_data = json.load(file)

    ground_truth_offsets = list(ghidra_data.keys())

    ground_truth_offsets = [int(i) for i in ground_truth_offsets]
    ground_truth_offsets = [x for x in ground_truth_offsets if text_section_offset <= x <= text_sextion_end]
    ground_truth_offsets.sort()
    return ground_truth_offsets



def get_ground_truth_objdump(exe_path, text_section_offset , text_section_len ):

    text_sextion_end = text_section_offset + text_section_len
    
    elf_file_name = os.path.basename(exe_path)
    objdump_file_path = os.path.join(ground_truth_path, elf_file_name.split('.')[0]) + '.json'
    
    with open(objdump_file_path, "r") as file:
        objdump_data = json.load(file)

    ground_truth_offsets = list(objdump_data["instructions"].keys())
    # print(ground_truth_offsets)
    if 'meta' in ground_truth_offsets:
        ground_truth_offsets.remove('meta')
    ground_truth_offsets = [int(i) for i in ground_truth_offsets]
    ground_truth_offsets = [x for x in ground_truth_offsets if text_section_offset <= x <= text_sextion_end]
    ground_truth_offsets.sort()

    
    return ground_truth_offsets

In [5]:


def find_data_in_textsection(ground_truth_offsets , text_section_offset , text_section_len, offset_inst_dict):

    for i in range(1, len(ground_truth_offsets)-1):
        distance = ground_truth_offsets[i+1] - ground_truth_offsets[i]

        inst_len = offset_inst_dict[ground_truth_offsets[i]].size 
        
        if distance!=inst_len:
            # print('offset_ranges[i]: ',ground_truth_offsets[i] , 'offset_ranges[i-1]: ',ground_truth_offsets[i-1], ' inst_len: ',inst_len  )
            
            print(ground_truth_offsets[i],' ' ,hex(ground_truth_offsets[i]) , offset_inst_dict[ground_truth_offsets[i]], ' len',offset_inst_dict[ground_truth_offsets[i]].size )
            print("\nByte GAP ###### ",distance ,' Missing bytes: ', distance - inst_len)
            
            for j in range( ground_truth_offsets[i] +inst_len , ground_truth_offsets[i+1]  ):
                if offset_inst_dict[j]:
                    print("# ",j, offset_inst_dict[j].mnemonic, offset_inst_dict[j].op_str , 'inst len:',offset_inst_dict[j].size )
                else:
                    print("# ",j, " invalid ")
            print('\n')
        else:
            print(ground_truth_offsets[i],' ', hex(ground_truth_offsets[i]) , offset_inst_dict[ground_truth_offsets[i]].mnemonic,offset_inst_dict[ground_truth_offsets[i]].op_str ,' len',offset_inst_dict[ground_truth_offsets[i]].size)
            pass

def linear_sweep(offset_inst , target_offset):
    inst_sequence = []
    max_seq_length = 15

    
    current_offset = target_offset
    for q in range(max_seq_length):

        if current_offset in offset_inst: #if end of text section
            current_instruction = offset_inst[current_offset]
            if current_instruction is None:

                inst_sequence.insert(0, None) 
                inst_sequence.append(None)
                return  inst_sequence
                
            current_offset = current_offset + current_instruction.size
            inst_sequence.append( str(current_instruction.address) +": "+ current_instruction.mnemonic +' '+ current_instruction.op_str )
            # inst_sequence.append( current_instruction)
            if current_instruction.mnemonic in ["ret", "jmp"]: #break linear sweep
                return inst_sequence

    return inst_sequence


for bin_file_path in bin_files:

    # try:
    #     ground_truth_offsets = get_ground_truth(bin_file_path, text_section_offset , len(textSection))
    # except Exception as e:
    #     print("An error occurred:", e)
    # print(ground_truth_offsets)
    
    
    md = Cs(CS_ARCH_X86, CS_MODE_64)
    md.detail = True
    offset_inst = {}

    # print("\n"*5)
    

    try:

        # PE
        pe_file = lief.parse(bin_file_path)
        
        # if rheader := pe.rich_header:
        #     print(rheader.key)



        # textSection = elffile.get_section_by_name('.text').data()
        # text_section_offset = elffile.get_section_by_name('.text')['sh_offset']


        text_section = pe_file.get_section(".text")


        # Get the section's offset in the file
        text_section_offset = text_section.pointerto_raw_data
    
        # Get the section's raw binary data
        textSection = bytes(text_section.content)
    



        
        ground_truth_offsets = get_ground_truth_ghidra(bin_file_path, text_section_offset , len(textSection))


    
        
    except Exception as e:
        print("An error occurred:", e ,bin_file_path)
    for byte_index in range(len(textSection)):
        
    
        try:    

            instruction = next(md.disasm(textSection[byte_index: byte_index+15 ], text_section_offset + byte_index ), None)
            
            # if instruction:
            #     print("%d:\t%s\t%s _\t%x" %(int(instruction.address), instruction.mnemonic, instruction.op_str, instruction.size))
            #     print(instruction.operands[0].type , instruction.operands[0].imm)
            #     # print( dir(instruction.operands[0] ))
      
            # else:
            #     print("%d:\t%s " % (text_section_offset + byte_index  , 'invalid instruction') )
                
            offset_inst[text_section_offset+byte_index] = instruction

        except Exception as e:
            print(traceback.print_exc() )
            print(e)

    
    
    offset_inst = collections.OrderedDict(sorted(offset_inst.items()))

    find_data_in_textsection(ground_truth_offsets , text_section_offset , len(textSection) , offset_inst)
    
    # print(offset_inst)
    for byte_index in range(len(textSection)):
        inst_seq = linear_sweep(offset_inst ,text_section_offset + byte_index )
        # print('\n','# # '*10)
        # print(inst_seq)
    # print(ground_truth_offsets)
    # print(offset_inst.keys())
    print('\n','# # '*30)


1028   0x404 lea rcx, [rip + 0x1fff5]  len 7
1035   0x40b call 0x480  len 5
1040   0x410 nop   len 1
1041   0x411 xor eax, eax  len 2
1043   0x413 add rsp, 0x28  len 4
1047   0x417 ret   len 1
1048   0x418 <CsInsn 0x418 [cc]: int3 >  len 1

Byte GAP ######  8  Missing bytes:  7
#  1049 int3  inst len: 1
#  1050 int3  inst len: 1
#  1051 int3  inst len: 1
#  1052 int3  inst len: 1
#  1053 int3  inst len: 1
#  1054 int3  inst len: 1
#  1055 int3  inst len: 1


1056   0x420 lea rax, [rip + 0x21c99]  len 7
1063   0x427 <CsInsn 0x427 [c3]: ret >  len 1

Byte GAP ######  9  Missing bytes:  8
#  1064 int3  inst len: 1
#  1065 int3  inst len: 1
#  1066 int3  inst len: 1
#  1067 int3  inst len: 1
#  1068 int3  inst len: 1
#  1069 int3  inst len: 1
#  1070 int3  inst len: 1
#  1071 int3  inst len: 1


1072   0x430 mov qword ptr [rsp + 0x20], r9  len 5
1077   0x435 mov qword ptr [rsp + 0x18], r8  len 5
1082   0x43a mov qword ptr [rsp + 0x10], rdx  len 5
1087   0x43f mov qword ptr [rsp + 8], rcx  

In [6]:
print('test')

test
